In [23]:
import pandas as pd
import os

In [24]:
csv_files = os.listdir('database')
dfs = []
for file in csv_files:
    file_path = os.path.join('database/', file)
    df = pd.read_csv(file_path, encoding='latin1', sep=';', skiprows=1)
    dfs.append(df)

data = pd.concat(dfs, ignore_index=True)
#    df = pd.read_csv(file_path, encoding='latin1', sep=';', skiprows=1, decimal=',')


In [25]:
data.head()

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2008,9,ADA MELLO,"Contratação de consultorias, assessorias, pesq...",NaN,NaN,NaN,NaN,NaN,"12351,52",2.008091e+12
1,2008,9,ADA MELLO,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,NaN,NaN,"386,6",2.008091e+12
2,2008,10,ADA MELLO,"Contratação de consultorias, assessorias, pesq...",NaN,NaN,NaN,NaN,NaN,"12351,52",2.008101e+12
3,2008,10,ADA MELLO,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,NaN,NaN,"2610,68",2.008101e+12
4,2008,11,ADA MELLO,"Contratação de consultorias, assessorias, pesq...",NaN,NaN,NaN,NaN,NaN,"12351,52",2.008111e+12


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317963 entries, 0 to 317962
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ANO                317963 non-null  int64  
 1   MES                317963 non-null  int64  
 2   SENADOR            317963 non-null  object 
 3   TIPO_DESPESA       317963 non-null  object 
 4   CNPJ_CPF           314819 non-null  object 
 5   FORNECEDOR         314819 non-null  object 
 6   DOCUMENTO          293153 non-null  object 
 7   DATA               314819 non-null  object 
 8   DETALHAMENTO       180631 non-null  object 
 9   VALOR_REEMBOLSADO  317963 non-null  object 
 10  COD_DOCUMENTO      317961 non-null  float64
dtypes: float64(1), int64(2), object(8)
memory usage: 26.7+ MB


In [27]:
data.memory_usage(deep=True).sort_values()

Index                     132
ANO                   2543704
MES                   2543704
COD_DOCUMENTO         2543704
DOCUMENTO            19575056
VALOR_REEMBOLSADO    19663920
DATA                 21193821
CNPJ_CPF             23681521
SENADOR              25264126
FORNECEDOR           29119901
DETALHAMENTO         36417929
TIPO_DESPESA         50407038
dtype: int64

In [28]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
ANO,317963.0,2.015154e+03,3.746778e+00,2008.0,2012.0,2015.0,2018.0,2.022000e+03
MES,317963.0,6.644276e+00,3.308155e+00,1.0,4.0,7.0,10.0,1.200000e+01
COD_DOCUMENTO,317961.0,1.985928e+10,1.987107e+11,143880.0,621197.0,2019630.0,2106125.0,2.009031e+12


## Removendo duplicatas e observações irrelevantes

In [29]:
data.drop_duplicates(inplace=True)

In [30]:
# sem útilidade ja que tem o _TIPO_DESPESA_ e tem muitos valores nulos
data.drop(columns='DETALHAMENTO', inplace=True)

In [31]:
# sem utilidade para os dados
data.drop(columns='DOCUMENTO', inplace=True)

In [32]:
# 2 valores nulos que eram irrelevantes comparado a totalidade dos dados
data.drop(columns = 'COD_DOCUMENTO',inplace=True)

## Fix Structural Errors

In [33]:
# Transformando coluna VALOR_REEMBOLSADO em float
data['VALOR_REEMBOLSADO'] = data['VALOR_REEMBOLSADO'].str.replace(',', '.').str.replace('\r\n', '', regex=True)
data['VALOR_REEMBOLSADO'] = data['VALOR_REEMBOLSADO'].astype(float)
data['VALOR_REEMBOLSADO'].dtype

In [35]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
ANO,317963.0,2015.153914,3.746778,2008.0,2012.0,2015.0,2018.0,2022.0
MES,317963.0,6.644276,3.308155,1.0,4.0,7.0,10.0,12.0
VALOR_REEMBOLSADO,317963.0,1031.730442,5682.034203,-1500.0,100.0,283.5,1000.0,2016557.0


In [46]:
# removendo dados negativos
data = data[data['VALOR_REEMBOLSADO'] >= 0]

In [36]:
# Verificando a relevancia dos dados nulos comparado com o total dos dados
data.isnull().sum() / data.shape[0] * 100

ANO                  0.000000
MES                  0.000000
SENADOR              0.000000
TIPO_DESPESA         0.000000
CNPJ_CPF             0.988794
FORNECEDOR           0.988794
DATA                 0.988794
VALOR_REEMBOLSADO    0.000000
dtype: float64

In [37]:
# Quantidade total de valores nulos
data.isnull().sum()

ANO                     0
MES                     0
SENADOR                 0
TIPO_DESPESA            0
CNPJ_CPF             3144
FORNECEDOR           3144
DATA                 3144
VALOR_REEMBOLSADO       0
dtype: int64

In [38]:
# Verificando se as colunas nulas estão ligadas
null_pattern_match = (data['CNPJ_CPF'].isnull() == data['FORNECEDOR'].isnull()) & (data['CNPJ_CPF'].isnull() == data['DATA'].isnull())
null_pattern_match.unique()

array([ True])

In [40]:
# Verificando a relevancia dos dados nulos de cada coluna comparado com o total dos dados
percentual_nulos_cnpj_cpf = data[data['CNPJ_CPF'].isnull()]['VALOR_REEMBOLSADO'].sum() / data[~data['VALOR_REEMBOLSADO'].isnull()]['VALOR_REEMBOLSADO'].sum() * 100
percentual_nulos_fornecedor = data[data['FORNECEDOR'].isnull()]['VALOR_REEMBOLSADO'].sum() / data[~data['FORNECEDOR'].isnull()]['VALOR_REEMBOLSADO'].sum() * 100
percentual_nulos_data = data[data['DATA'].isnull()]['VALOR_REEMBOLSADO'].sum() / data[~data['DATA'].isnull()]['VALOR_REEMBOLSADO'].sum() * 100

print(f'cnpj_cpf = {percentual_nulos_cnpj_cpf :.2f}%\nfornecedor = {percentual_nulos_fornecedor:.2f}%\ndata = {percentual_nulos_data:.2f}%')

cnpj_cpf = 4.38%
fornecedor = 4.58%
data = 4.58%


In [41]:
# Conforme analisado, é a relacao entres os valores nulos e nao nulos é muito baixa.
data.dropna(subset =['CNPJ_CPF', 'FORNECEDOR', 'DATA'], inplace=True)

In [42]:
data.isnull().sum()

ANO                  0
MES                  0
SENADOR              0
TIPO_DESPESA         0
CNPJ_CPF             0
FORNECEDOR           0
DATA                 0
VALOR_REEMBOLSADO    0
dtype: int64

In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 314816 entries, 2449 to 317962
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ANO                314816 non-null  int64  
 1   MES                314816 non-null  int64  
 2   SENADOR            314816 non-null  object 
 3   TIPO_DESPESA       314816 non-null  object 
 4   CNPJ_CPF           314816 non-null  object 
 5   FORNECEDOR         314816 non-null  object 
 6   DATA               314816 non-null  object 
 7   VALOR_REEMBOLSADO  314816 non-null  float64
dtypes: float64(1), int64(2), object(5)
memory usage: 21.6+ MB


In [52]:
data.shape

(314816, 8)